In [18]:
import ray

ray.init("ray://10.128.0.44:10001",
         runtime_env={
             "pip": [
                 "-f 'jax[tpu]==0.4.11' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html"
             ],
         })

RuntimeError: Ray Client is already connected. Maybe you called ray.init("ray://<address>") twice by accident?

Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::MyActor.say_hello() (pid=20581, ip=10.130.0.3, actor_id=b11b054ffaec5506a82e2bb103000000, repr=<__main__.MyActor object at 0x7f1b19eb46a0>)
TypeError: say_hello() takes 0 positional arguments but 1 was given


In [28]:
@ray.remote(resources={"TPU": 1})
def hello_world():
    print("test")
    return "hello world"

remote = hello_world.remote()
print(ray.get(remote))

(autoscaler +52m48s) Adding 1 node(s) of type ray_tpu.
hello world


(hello_world pid=20385, ip=10.130.0.5) test
(autoscaler +55m8s) Resized to 481 CPUs.
(autoscaler +55m8s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'TPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(autoscaler +1h58s) Removing 1 nodes of type ray_tpu (idle).
(autoscaler +1h1m9s) Resized to 241 CPUs.


The autoscaler failed with the following error:
Terminated with signal 15
  File "/home/ubuntu/.local/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 720, in <module>
    monitor.run()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 595, in run
    self._run()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 449, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)

2023-08-03 21:00:23,753	WARNING dataclient.py:403 -- Encountered connection issues in the data channel. Attempting to reconnect.
2023-08-03 21:00:53,960	WARNING dataclient.py:410 -- Failed to reconnect the data channel


In [20]:
@ray.remote(resources={"TPU": 1})
class MyActor:
    def return_hello(self):
        return "hello"
    
    def return_goodbye(self):
        return "goodbye"

    def say_hello(self):
        print("hello")


In [21]:
actor = MyActor.remote()

In [22]:
hello_handle = actor.say_hello.remote()

(MyActor pid=21629, ip=10.130.0.2) hello


In [17]:
print(ray.get(hello_handle))

hello


In [24]:
@ray.remote(resources={"TPU": 1})
def my_ml_workload():
    import jax
    return jax.device_count()

In [27]:
ray.get(my_ml_workload.remote())

4

(raylet, ip=10.130.0.4) [2023-08-03 20:12:19,467 E 22561 22626] (raylet) agent_manager.cc:135: The raylet exited immediately because the Ray agent failed. The raylet fate shares with the agent. This can happen because the Ray agent was unexpectedly killed or failed. Agent can fail when
(raylet, ip=10.130.0.4) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet, ip=10.130.0.4) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/dashboard_agent.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure.
(raylet, ip=10.130.0.4) - The agent is killed by the OS (e.g., out of memory).


(autoscaler +22m51s) Resized to 481 CPUs.
(autoscaler +28m41s) Removing 1 nodes of type ray_tpu (idle).
(autoscaler +28m51s) Resized to 241 CPUs.
